In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Lecture du fichier CSV
data = pd.read_csv("ETTh1_without_missing.csv")

# Sélection des caractéristiques nécessaires pour l'entraînement
OT_values = data["OT"].values

# Fonction pour créer les séquences de données
def create_sequences(values, sequence_length):
    X, y = [], []
    for i in range(len(values) - sequence_length - 100):
        X.append(values[i:i + sequence_length])
        y.append(values[i + sequence_length:i + sequence_length + 100])
    return np.array(X), np.array(y)

# Longueur de la séquence
SEQUENCE_LENGTH = 100

# Création des séquences pour l'entraînement du modèle
X_train, y_train = create_sequences(OT_values, SEQUENCE_LENGTH)

# # Création du modèle LSTM avec TensorFlow
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(64, input_shape=(SEQUENCE_LENGTH, 1)),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(100)  # Un seul neurone de sortie pour prédire la valeur suivante
# ])

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(SEQUENCE_LENGTH, 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100)  # Neurone de sortie pour prédire la valeur suivante
])

# Compilation du modèle
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["mae"])

# Entraînement du modèle
model.fit(X_train, y_train, epochs= 20, batch_size=32)

# Prédiction des 100 valeurs suivantes
last_sequence = OT_values[-SEQUENCE_LENGTH:].reshape(1, SEQUENCE_LENGTH, 1)
predicted_values = model.predict(last_sequence).reshape(-1)

# Sélection des 100 premières valeurs prédites disponibles
num_predicted_values = min(len(predicted_values), 100)
predicted_df = pd.DataFrame({'id': range(num_predicted_values), 'OT': predicted_values[:num_predicted_values]})

# Écriture dans un nouveau fichier CSV
predicted_df.to_csv('OT_predictions.csv', index=False)


Epoch 1/20
535/535 [==============================] - 34s 45ms/step - loss: 34.0733 - mae: 3.8989
Epoch 2/20
535/535 [==============================] - 24s 45ms/step - loss: 12.8067 - mae: 2.7404
Epoch 3/20
535/535 [==============================] - 24s 45ms/step - loss: 12.3199 - mae: 2.6757
Epoch 4/20
535/535 [==============================] - 22s 40ms/step - loss: 12.0102 - mae: 2.6358
Epoch 5/20
535/535 [==============================] - 20s 36ms/step - loss: 11.5945 - mae: 2.5890
Epoch 6/20
535/535 [==============================] - 21s 39ms/step - loss: 11.2196 - mae: 2.5444
Epoch 7/20
535/535 [==============================] - 22s 42ms/step - loss: 10.9470 - mae: 2.5112
Epoch 8/20
535/535 [==============================] - 24s 45ms/step - loss: 10.6262 - mae: 2.4747
Epoch 9/20
535/535 [==============================] - 24s 46ms/step - loss: 10.3966 - mae: 2.4476
Epoch 10/20
535/535 [==============================] - 25s 46ms/step - loss: 10.0735 - mae: 2.4044
Epoch 11/20
535/535